# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using AtomsBuilder
using DFTK
using LinearAlgebra
using PseudoPotentialData

pseudopotentials = PseudoFamily("dojo.nc.sr.pbesol.v0_4_1.standard.upf")
model = model_DFT(bulk(:Si); functionals=PBEsol(), pseudopotentials)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.397775552065                   -0.90    5.2   28.3ms
  2   -8.400242081302       -2.61       -1.74    1.0   19.1ms
  3   -8.400402488477       -3.79       -2.95    1.5   43.1ms
  4   -8.400427813289       -4.60       -2.93    3.2   25.2ms
  5   -8.400427831480       -7.74       -2.93    1.0   19.1ms
  6   -8.400428145807       -6.50       -4.62    1.0   19.1ms
  7   -8.400428155591       -8.01       -4.32    3.2   25.6ms
  8   -8.400428156253       -9.18       -5.55    1.0   19.5ms
  9   -8.400428156276      -10.64       -6.29    2.0   39.9ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -8.397732847592                   -0.90           4.8    468ms
  2   -8.400362208716       -2.58       -1.77   0.80    2.2    499ms
  3   -8.400419250268       -4.24       -2.96   0.80    1.0    173ms
  4   -8.400428102119       -5.05       -3.31   0.80    3.0   75.9ms
  5   -8.400428151633       -7.31       -4.42   0.80    1.2   16.9ms
  6   -8.400428156239       -8.34       -5.45   0.80    3.0   21.3ms
  7   -8.400428156276      -10.44       -6.26   0.80    2.2   19.5ms


## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   +0.946308383213                   -1.07    2.77s
  2   -1.365211624405        0.36       -0.68    107ms
  3   -3.525257210901        0.33       -0.41   38.9ms
  4   -3.940299491671       -0.38       -0.47   39.1ms
  5   -6.031596312734        0.32       -0.48   64.6ms
  6   -6.765559486046       -0.13       -0.71   38.8ms
  7   -7.760941373224       -0.00       -0.87   38.9ms
  8   -7.910924858146       -0.82       -1.45   28.7ms
  9   -8.113458605381       -0.69       -1.67   45.8ms
 10   -8.186798478914       -1.13       -1.79   29.1ms
 11   -8.309694100798       -0.91       -1.97   28.7ms
 12   -8.355554975426       -1.34       -2.20   28.7ms
 13   -8.377787481484       -1.65       -2.15   29.1ms
 14   -8.389731347973       -1.92       -2.47   28.6ms
 15   -8.393128662751       -2.47       -2.44   33.1ms
 16   -8.397123839046       -2.40       -2.71   28.8ms
 17   -8.39

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.397797240772                   -0.90    5.0   26.4ms


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -8.400427970868                   -1.78    24.3s
  2   -8.400428156277       -6.73       -4.02    2.15s
  3   -8.400428156277      -14.45       -7.79    125ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 6.635300380031439e-7
|ρ_newton - ρ_scfv| = 2.0390101233108131e-7
|ρ_newton - ρ_dm|   = 1.109330286214458e-6
